In [22]:
import modules.swmm_ex.swmm_ex_multiprocessing as ex
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
import os
from modules.extract_events import extract_events
from modules.swmm_ex.swmm_ex_multiprocessing import swmm_mp

# Set relevant data paths
events_path = os.path.join('02_input_data','events_FMO.csv')
P_path = os.path.join('02_input_data','P_FMO.csv')
save_folder = os.path.join('03_sim_data','quick_sim')
# events_path = '02_input_data\\events_FMO.csv'
# P_path = '02_input_data\\P_FMO.csv'
# save_folder = '03_sim_data\\quick_sim'

# Read in data
P_events = pd.read_csv(events_path)
P_series = pd.read_csv(P_path)


P_sample = P_events[(P_events['hN_mm'] < 2.5) & (P_events['hN_mm'] > 0.5)].sample(5,random_state=5)

P_sample

,start,end,hN_mm,iN_mean,iN_max,hN_max_60,DN_h
441,2016-05-14 14:55:00,2016-05-14 15:35:00,0.95,0.105556,0.59,0.00,0.666667
717,2017-12-15 10:20:00,2017-12-15 10:55:00,0.80,0.100000,0.27,0.00,0.583333
428,2016-04-24 09:00:00,2016-04-24 11:45:00,1.16,0.034118,0.56,0.68,2.750000
1273,2021-04-12 13:35:00,2021-04-12 17:25:00,0.99,0.021064,0.33,0.77,3.833333
1357,2021-10-05 16:05:00,2021-10-05 17:20:00,0.94,0.058750,0.12,0.76,1.250000


In [23]:
len(P_events[(P_events['hN_mm'] <= 1) & (P_events['hN_mm'] > 0.5)])

267

In [24]:
extract_events(P_sample, P_series, save_folder)

Event 2016-05-14 14:55:00 saved to 03_sim_data\quick_sim\2016-05-14 14 55 00_hN0.95.csv
Event 2017-12-15 10:20:00 saved to 03_sim_data\quick_sim\2017-12-15 10 20 00_hN0.8.csv
Event 2016-04-24 09:00:00 saved to 03_sim_data\quick_sim\2016-04-24 09 00 00_hN1.16.csv
Event 2021-04-12 13:35:00 saved to 03_sim_data\quick_sim\2021-04-12 13 35 00_hN0.99.csv
Event 2021-10-05 16:05:00 saved to 03_sim_data\quick_sim\2021-10-05 16 05 00_hN0.94.csv


In [1]:
import swmm_api
from swmm_api.input_file import section_labels as sections
from modules.inp_helpers import euler_to_inp, event_to_inp

# get current path of working directory
current_path = os.getcwd()
###########################################################################################################
# Input section
## Input parameters for inp file generation
base_inp_path = os.path.join('03_sim_data', 'Gievenbeck_20240325.inp')
# base_inp_path = '03_sim_data\\Gievenbeck_20240325.inp'
# path to folder with rain event data
event_data_path = save_folder
# path to kostra data
kostra_data_path = os.path.join(current_path, '02_input_data','kostra_118111.csv')
# set maximum duration time [min] for Kostra data
max_duration = 72*60
# Name of the study area
name_place = 'Gievenbeck'
# Path to save the inp files
save_inp_path = save_folder
# Euler type for Kostra data (2 is standard)
euler_typ = 2
# Start time of the simulation
start_time = pd.to_datetime('2024-01-01 00:00')
# Buffer time before and after the rainfall event
buffer_time = pd.Timedelta('2h')
# Name of the Kostra time series to be included in the inp file
TSnameKostra = 'Kostra'
# Name of the measured time series to be included in the inp file
TSnameEvent = 'FMO'
# Time interval of the time series in minutes
TSinterval = 5

########################
# read kostra data
kostra = pd.read_csv(kostra_data_path, delimiter=',', index_col=0)
# get return preiods and durations from kostra table
returnrate = kostra.columns.astype(int)
# duration needs to be larger than 15min
durations = kostra.index[(kostra.index >= 15) & (kostra.index <= max_duration)]
# calculate end time of the simulation with start time and buffer time
end_time = start_time + pd.Timedelta(minutes=int(max(durations))) + buffer_time * 2

########################
inp_base = swmm_api.read_inp_file(base_inp_path)
# Update OPTIONS of inp file
inp_base['OPTIONS'].update({'START_DATE': start_time.date()})
inp_base['OPTIONS'].update({'START_TIME': start_time.time()})
inp_base['OPTIONS'].update({'REPORT_START_DATE': start_time.date()})
inp_base['OPTIONS'].update({'REPORT_START_TIME': start_time.time()})
inp_base['OPTIONS'].update({'END_DATE': end_time.date()})
inp_base['OPTIONS'].update({'END_TIME': end_time.time()})
# inp_base['OPTIONS'].update({'THREADS': cpu_cores})

ModuleNotFoundError: No module named 'swmm_api'

In [26]:
inp = inp_base
try:
    del inp[sections.TIMESERIES][TSnameKostra]
except Exception:
    pass
try:
    del inp['RAINGAGES'][TSnameKostra]
except Exception:
    pass

for file_name in os.listdir(event_data_path):
    if file_name.endswith('.csv'):
        
        file_path = os.path.join(event_data_path, file_name)
        event_data = pd.read_csv(file_path)
        inp = event_to_inp(inp, event_data, start_time=start_time + buffer_time, TSname=TSnameEvent)
        for subcatchment in inp['SUBCATCHMENTS']:
            inp['SUBCATCHMENTS'][subcatchment].rain_gage = TSnameEvent
        file_name = file_name.replace('.csv', '')
        file_name = file_name.replace('.', ' ')
        inp['TITLE'] = f'{name_place}_{file_name}'
        inp.write_file(os.path.join(save_inp_path,f'{name_place}_{file_name}.inp'))

In [27]:
swmm_mp(save_folder)




Total simulation time:  10.417437553405762
All done


In [28]:
len(P_events[(P_events['hN_mm'] <= 1) & (P_events['hN_mm'] > 0.5)])

267

#### Simulation: Gievenbeck_2016-05-14 14 55 00_hN0 95
Niederschlagshöhe: 0.95 mm
Abflusshöhe: 0.064 mm

#### Simulation: Gievenbeck_2021-04-12 13 35 00_hN0 99
Niederschlagshöhe: 0.99 mm
Abflusshöhe: 0.051 mm

#### Simulation: Gievenbeck_2016-10-28 14 50 00_hN1 47
Niederschlagshöhe: 1.47 mm
Abflusshöhe: 0.109 mm

#### Simulation: Gievenbeck_2016-07-12 17 15 00_hN1 61
Niederschlagshöhe: 1.61 mm
Abflusshöhe: 0.130 mm

Damit die Niederschlagsereignisse eingegrenzt werden können, werden Ereignisse mit weniger als 1 mm Niederschlag als insignifikant eingeordnet und nicht berücksichtigt.
Dadurch werden 267 potentielle Ereignisse entfernt. 
Stichprobensimulationen zeigten, dass mit ca. 1 mm Niederschlaghöhe weniger als 0,1 mm Abfluss erzeugt wurden.